In [1]:
import sys
import os
import pandas as pd
from binance.client import Client
from keys import BINANCE_API_KEY, BINANCE_SECRET_KEY

sys.path.append('..')

path = "./data"

if not os.path.exists(path):
    os.mkdir(os.path.join(path))
    
client = Client(api_key=BINANCE_API_KEY, api_secret=BINANCE_SECRET_KEY)


In [2]:
def write_klines_to_csv(symbol_, interval_, year):
    path = "./data/" + str(year)
    if not os.path.exists(path):
        os.mkdir(os.path.join(path))

    for i in range(1, 13):
        start_str_ = str(year) + '-' + str(i).zfill(2) + '-01'
        end_str_ = str(year + 1) + '-' + str((i + 1) % 12).zfill(2) + '-01' if (i + 1) % 12 == 1 else str(year) + '-' + str((i + 1)).zfill(2) + '-01'

        if not os.path.exists(os.path.join(path + '/klinesETH' + str(year) + str(i).zfill(2) + '.csv')):
            klines = client.get_historical_klines(symbol=symbol_, interval=interval_, start_str=start_str_, end_str=end_str_)

            if not klines:
                print(f"No data for {start_str_} to {end_str_}")
                continue

            df = pd.DataFrame(klines).iloc[:, :6]
            
            if df.empty or not all(col in df.columns for col in range(6)):
                print("DataFrame does not have the expected columns.")
                continue
            
            df['timestamp'] = df[0]
            df['timestamp_human'] = pd.to_datetime(df[0], unit='ms')
            df['open'] = df[1].astype(float)
            df['high'] = df[2].astype(float)
            df['low'] = df[3].astype(float)
            df['close'] = df[4].astype(float)
            df['volume'] = (df[5].astype(float) * df[4].astype(float)).astype(int)
            df = df.drop([0, 1, 2, 3, 4, 5], axis=1)
            df = df.drop(df.index[-1], errors='ignore')

            df.to_csv(os.path.join(path + '/klinesETH' + str(year) + str(i).zfill(2) + '.csv'))

In [3]:
def struct_by_year(start_year, end_year):
    for i in range(start_year, end_year + 1):
        write_klines_to_csv('ETHUSDT', '1s', i)

In [4]:
struct_by_year(2017, 2022)

No data for 2017-01-01 to 2017-02-01
No data for 2017-02-01 to 2017-03-01
No data for 2017-03-01 to 2017-04-01
No data for 2017-04-01 to 2017-05-01
No data for 2017-05-01 to 2017-06-01
No data for 2017-06-01 to 2017-07-01
No data for 2017-07-01 to 2017-08-01
